In [ ]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv  

load_dotenv()

In [10]:
KEY = os.getenv("OPENAI_API_KEY", None)
if not KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

ValueError: Please set the OPENAI_API_KEY environment variable.

In [11]:
from langchain.chat_models import ChatOpenAI

llms = ChatOpenAI(opneai_ai_key=KEY,
                  model_name="gpt-3.5-turbo",temperature=0.5)

C:\Users\yasha\AppData\Local\Temp\ipykernel_13588\31586235.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llms = ChatOpenAI(opneai_ai_key=KEY,
WARNING! opneai_ai_key is not default parameter.
                    opneai_ai_key was transferred to model_kwargs.
                    Please confirm that opneai_ai_key is what you intended.


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_name': 'gpt-3.5-t...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error